 

# 使用RNN进行情绪分析

在这本笔记本中，您将实施执行情绪分析的循环神经网络。使用RNN而不是feedfoward网络更准确，因为我们可以包括有关*序列*的信息。在这里，我们将使用电影评论的数据集，并附有标签。

该网络的架构如下所示。

<img src="assets/network_diagram.png" width=400px>

在这里，我们将传入一个嵌入层。我们需要一个嵌入层，因为我们有成千上万的单词，所以我们需要一个比一个热编码向量更有效的输入数据表示。您应该已经从word2vec课程中看到过。实际上你可以用word2vec来训练一个嵌入，并在这里使用它。但是，只要拥有一个嵌入层，让网络学习嵌入表就可以了。

从嵌入层，新的表示将被传递给LSTM单元。这些将添加到网络的重复连接，因此我们可以包括关于数据中单词序列的信息。最后，LSTM单元将在这里进入S形输出层。我们正在使用sigmoid，因为我们试图预测这个文本是否具有正面或负面的情绪。输出层只是一个单位，然后是S形激活功能。

我们不关心Sigmoid输出，除了最后一个，我们可以忽略其余的。我们将从最后一步的输出和培训标签计算成本。

In [1]:
import numpy as np
import tensorflow as tf

In [3]:
with open('reviews.txt', 'r') as f:
    reviews = f.read()
with open('labels.txt', 'r') as f:
    labels = f.read()

In [4]:
reviews[:2000]

'bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   \nstory of a man who has unnatural feelings for a pig . starts out with a opening scene that is a terrific example of absurd comedy . a formal orchestra audience is tu

## 数据预处理

构建神经网络模型的第一步是将数据转化为正确的形式进入网络。 由于我们使用嵌入层，我们需要用一个整数对每个单词进行编码。 我们也想清理一下。

您可以看到上面的评论数据的例子。 我们想摆脱那些时期。 另外，您可能会注意到，这些评论用换行符“\ n”分隔。 为了处理这些问题，我将使用`\ n`作为分隔符将文本分成每个评论。 然后我可以将所有的评论结合成一个大字符串。

首先，我们删除所有的标点符号。 然后获取所有没有换行符的文本，并将其分成单个单词。

In [7]:
"""
string模块中可以使用的全局变量
>>> import string
>>> string.digits
'0123456789'
>>> string.hexdigits
'0123456789abcdefABCDEF'
>>> string.octdigits
'01234567'
>>> string.letters
'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
>>> string.lowercase
'abcdefghijklmnopqrstuvwxyz'
>>> string.uppercase
'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
>>> string.printable
'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c'
>>> string.punctuation
'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
>>> string.whitespace
'\t\n\x0b\x0c\r '
>>>
"""
from string import punctuation
all_text = ''.join([c for c in reviews if c not in punctuation])
reviews = all_text.split('\n')

all_text = ' '.join(reviews)
words = all_text.split()

In [8]:
all_text[:2000]

'bromwell high is a cartoon comedy  it ran at the same time as some other programs about school life  such as  teachers   my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers   the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students  when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled          at           high  a classic line inspector i  m here to sack one of your teachers  student welcome to bromwell high  i expect that many adults of my age think that bromwell high is far fetched  what a pity that it isn  t    story of a man who has unnatural feelings for a pig  starts out with a opening scene that is a terrific example of absurd comedy  a formal orchestra audience is turned into an insane  violent m

In [6]:
words[:100]

['bromwell',
 'high',
 'is',
 'a',
 'cartoon',
 'comedy',
 'it',
 'ran',
 'at',
 'the',
 'same',
 'time',
 'as',
 'some',
 'other',
 'programs',
 'about',
 'school',
 'life',
 'such',
 'as',
 'teachers',
 'my',
 'years',
 'in',
 'the',
 'teaching',
 'profession',
 'lead',
 'me',
 'to',
 'believe',
 'that',
 'bromwell',
 'high',
 's',
 'satire',
 'is',
 'much',
 'closer',
 'to',
 'reality',
 'than',
 'is',
 'teachers',
 'the',
 'scramble',
 'to',
 'survive',
 'financially',
 'the',
 'insightful',
 'students',
 'who',
 'can',
 'see',
 'right',
 'through',
 'their',
 'pathetic',
 'teachers',
 'pomp',
 'the',
 'pettiness',
 'of',
 'the',
 'whole',
 'situation',
 'all',
 'remind',
 'me',
 'of',
 'the',
 'schools',
 'i',
 'knew',
 'and',
 'their',
 'students',
 'when',
 'i',
 'saw',
 'the',
 'episode',
 'in',
 'which',
 'a',
 'student',
 'repeatedly',
 'tried',
 'to',
 'burn',
 'down',
 'the',
 'school',
 'i',
 'immediately',
 'recalled',
 'at',
 'high']

### 编码单词

嵌入式查找要求我们将整数传递给我们的网络。 最简单的方法是创建将词表中的单词映射为整数的字典。 然后我们可以将每个评论转换成整数，以便将它们传递到网络中。

> **练习：**现在你要用整数对单词进行编码。 构建一个将单词映射到整数的字典。 稍后我们将使用零填充输入向量，因此确保整数**从1开始，而不是0 **。
>此外，将评论转换为整数，并将评论存储在名为“comments_ints”的新列表中。

In [15]:
from collections import Counter
#吧所有的单词计算词频
counts = Counter(words)
#找出所有的单词，
vocab = sorted(counts,key=counts.get,reverse=True)
#循环每个单词，找到单词对应的词频数据
vocab_to_int = {word:li for li ,word in enumerate(vocab,1)}

reviews_ints = []
for each in reviews:
    reviews_ints.append([vocab_to_int[word] for word in each.split()])

print(reviews_ints[100])

[10, 168, 8, 37, 472, 12, 59, 101, 31, 132, 154, 69, 26, 131, 642, 44, 11, 18, 530, 11, 18, 285, 23, 12, 76, 42, 326, 78, 576, 23, 59, 1394, 5, 724, 44, 8, 10, 1248, 499, 11, 20, 66284, 2, 131, 80, 10, 474, 11, 20, 215, 23, 16, 295, 19, 8, 430, 120, 1191, 1, 3348, 4, 109, 7, 7, 15, 16, 147, 12, 102, 12, 11, 20, 6, 3837, 42, 119, 5468, 72, 69, 64, 28, 1, 18, 169, 1027, 5, 1256, 16, 12, 3962, 21, 66, 8, 13, 24, 1384, 7605, 891, 12, 95, 3, 20, 653, 42, 3, 557, 17, 1, 797, 18, 169, 1027, 78, 95, 1, 20, 3, 653, 1000, 17, 2558, 295, 14, 9, 4221, 278, 13, 386, 8, 3, 20, 39, 11, 68, 114, 77, 91, 160, 32, 219, 24, 17, 1, 542, 4, 315, 298, 871, 5, 65, 1819, 1, 3292, 4, 1, 1552, 165, 6276, 11, 20, 6, 37, 1839, 12, 59, 566, 3659, 8167, 1, 764, 6276, 4, 1, 1552, 9, 1, 20, 17, 1, 146, 6276, 12, 11078, 245, 9, 4237, 59, 1, 23960, 16, 4070, 20492, 4679, 7, 7, 201, 11, 18, 3, 967, 21, 479, 1, 1384, 195, 11, 18, 61, 4759, 192, 57, 54, 11, 18, 165, 379, 45, 2, 78, 635, 2202, 34, 665, 4, 1610, 62, 6, 1, 1

### 编码标签

我们的标签是“正面”或“负面”。 要在我们的网络中使用这些标签，我们需要将它们转换为0和1。

> **练习：**将标签从“正”和“负”分别转换为1和0。

In [16]:
# Convert labels to 1s and 0s for 'positive' and 'negative'
labels = labels.split('\n')
labels = np.array([1 if each =='positive' else 0 for each in labels])

如果你正确地构建了`标签'，你应该看到下一个输出。

In [17]:
from collections import Counter
review_lens = Counter([len(x) for x in reviews_ints])
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))

Zero-length reviews: 1
Maximum review length: 2514


 
好的，这里有几个问题。 我们似乎有一个零长度的审查。 而且，我们的RNN的最大审查长度是太多的步骤。 让我们截断到200步。 对于小于200的评论，我们会用0秒。 对于超过200次的评论，我们可以将其截断为前200个字符。

> **练习：**首先，从`comments_ints`列表中删除零长度的评论。

In [18]:
non_zero_idx = [li for li ,review in enumerate(reviews_ints) if len(review)!=0 ]
len(non_zero_idx)

25000

In [21]:
# Filter out that review with 0 length
reviews_ints = [reviews_ints[ii] for ii in non_zero_idx]
labels = np.array([labels[ii] for ii in non_zero_idx])
print (len(reviews_ints))
print (len(labels))

25000
25000


> **练习：**现在，创建一个包含我们传递给网络的数据的数组“features”。 数据应该来自`review_ints`，因为我们想把整数提供给网络。 每行应该是200元素长。 对于短于200个字的评论，左键为0。 也就是说，如果审查是“[最好的”，“电影”，“永远”]，“[117，18，128]”作为整数，行将看起来像`[0，0，0，.. 。，0，117，18，128] 对于超过200的评论，使用前200个字作为特征向量。

这不是微不足道的，有一些方法可以做到这一点。 但是，如果您要建立自己的深入学习网络，那么您将不得不习惯于准备数据。



In [22]:
seq_len = 200
features = np.zeros((len(reviews_ints),seq_len),dtype=int)
for i,row in enumerate(reviews_ints):
    features[i, -len(row):] = np.array(row)[:seq_len]

如果您正确构建功能，它应该像下面的单元格输出。

In [23]:
features[:10,:100]

array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0, 22089,   308,     6,
            3,  1050,   207,     8,  2143,    32,     1,   171,    57,
           15,    49,    81,  5851,    44,   382,   110,   140,    15,
         5240,    60,   154,     9,     1,  5007,  5879,   475,    71,
            5,   260,    12, 22089,   308,    13,  1979,     6,    74,
         2406],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     

## Training, Validation, Test



我们的数据形状很好，我们将其分为培训，验证和测试集。

> **练习：**在这里创建训练，验证和测试集。 您需要为功能和标签创建集合，例如`train_x`和`train_y`。 定义一个分数分数，“split_frac”作为保留在训练集中的数据的一部分。 通常设定为0.8或0.9。 剩下的数据将被分成两半，以创建验证和测试数据。

In [26]:
split_frac = 0.8
split_index = int(len(features)*split_frac)
train_x, val_x = features[:split_index],features[split_index:]
train_y, val_y = labels[:split_index],labels[split_index:]

test_index = int(len(val_x)*0.5)
val_x, test_x = val_x[:test_index],val_x[test_index:]
val_y, test_y = val_y[:test_index],val_y[test_index:]


print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

			Feature Shapes:
Train set: 		(20000, 200) 
Validation set: 	(2500, 200) 
Test set: 		(2500, 200)


具有0.8,0.1,0.1的训练，验证和文本分数，最终形状应如下所示：
```
                    Feature Shapes:
Train set: 		 (20000, 200) 
Validation set: 	(2500, 200) 
Test set: 		  (2500, 200)
```

## 建立图表

在这里，我们将构建图。 首先，定义超参数。

*`lstm_size`：LSTM单元格中隐藏层中的单元数。 通常更大的是更好的性能明智。 常用值为128,256,512等
*`lstm_layers`：网络中的LSTM层数。 我会从1开始，然后添加更多，如果我不适合。
*`batch_size`：在一个培训通行证中提供网络的评论数量。 通常，这应该设置为尽可能高，没有内存不足。
*“learning_rate”：学习率

In [27]:
lstm_size = 256
lstm_layers = 1
batch_size = 500
learning_rate = 0.001

对于网络本身，我们将通过我们的200个元素长的评估向量。 每批将是“batch_size”向量。 我们还将在LSTM层上使用dropout，所以我们将为保留概率创建一个占位符。

> **练习：**创建`inputs_`，`labels_`，并使用`tf.placeholder`退出`keep_prob`占位符。 `labels_`需要二维以后才能使用某些功能。 因为`keep_prob`是一个标量（一个0维张量），所以你不应该为`tf.placeholder`提供一个大小。

In [28]:
n_words = len(vocab_to_int) + 1 # Adding 1 because we use 0's for padding, dictionary started at 1

# Create the graph object
graph = tf.Graph()
# Add nodes to the graph
with graph.as_default():
    inputs_ = tf.placeholder(tf.int32,[None,None],name='inputs')
    labels_ = tf.placeholder(tf.int32,[None,None],name='labels')
    keep_prob = tf.placeholder(tf.float32,name='keep_prob')

### 嵌入

现在我们将添加一个嵌入层。我们需要这样做，因为我们的词汇中有74000个单词。在这里对我们的课程进行一次热编码是非常低效的。你应该记住从word2vec课程处理这个问题。而不是单热编码，我们可以有一个嵌入层，并将该层用作查找表。您可以使用word2vec训练一个嵌入层，然后将其加载到此处。但是，只需创建一个新层，让网络学习权重就可以了。

> **练习：**将嵌入式查找矩阵创建为“tf.Variable”。使用该嵌入矩阵获取嵌入的向量以[`tf.nn.embedding_lookup`]（https://www.tensorflow.org/api_docs/python/tf/nn/embedding_lookup ）传递给LSTM单元格。该函数采用嵌入矩阵和输入张量，如审查向量。然后，它将返回另一个与嵌入向量的张量。因此，如果嵌入层有200个单位，则函数将返回尺寸[batch_size，200]的张量。



In [29]:
# Size of the embedding vectors (number of units in the embedding layer)
embed_size = 300 

with graph.as_default():
    embedding = tf.Variable(tf.random_uniform((n_words, embed_size),-1,1))
    embed = tf.nn.embedding_lookup(embedding,inputs_)


### LSTM单元格

<img src="assets/network_diagram.png" width=400px>

接下来，我们将创建我们的LSTM单元格，用于经常性网络（[TensorFlow文档]（https://www.tensorflow.org/api_docs/python/tf/contrib/rnn ））。这里我们只是定义单元格的样子。这不是实际构建图，只是在图中定义我们想要的单元格的类型。

要为图形创建一个基本的LSTM单元格，您需要使用`tf.contrib.rnn.BasicLSTMCell`。查看功能文档：

```
tf.contrib.rnn.BasicLSTMCell（num_units，forget_bias = 1.0，input_size = None，state_is_tuple = True，activation = <function tanh at 0x109f1ef28>）
```

您可以看到它在该代码中使用称为“num_units”的参数，单元格中的单位数，称为“lstm_size”。那么，你可以写一些类似的东西

```
lstm = tf.contrib.rnn.BasicLSTMCell（num_units）
```

用“num_units”创建一个LSTM单元格。接下来，您可以使用`tf.contrib.rnn.DropoutWrapper`向单元格添加退出。这只是将单元格包装在另一个单元格中，但是将输出和/或输出添加到输出中。这是一个非常方便的方式，使您的网络更好，几乎没有任何努力！所以你会做类似的事情

```
drop = tf.contrib.rnn.DropoutWrapper（cell，output_keep_prob = keep_prob）
```

大多数情况下，您的网络将具有更好的性能，更多的层。这是深入学习的魔力，增加更多的层次可以让网络学习真正复杂的关系。再次，有一个简单的方式来创建具有`tf.contrib.rnn.MultiRNNCell`的多层LSTM单元格：

```
cell = tf.contrib.rnn.MultiRNNCell（[drop] * lstm_layers）
```

这里，`[drop] * lstm_layers`创建一个长度为lstm_layers的单元格列表（`drop`）。 “MultiRNNCell”包装器将其构建到多个RNN单元格中，一个用于列表中的每个单元格。

因此，您在网络中使用的最后一个单元格实际上是多个（或只有一个）具有删除的LSTM单元格。但是，从建筑的角度来看，这一切都是一样的，只是一个更复杂的单元格图形。

> **练习：**下面，使用`tf.contrib.rnn.BasicLSTMCell`创建一个LSTM单元格。然后，用`tf.contrib.rnn.DropoutWrapper`添加它。最后，使用`tf.contrib.rnn.MultiRNNCell`创建多个LSTM图层。

这是一个关于建立RNN的教程（https://www.tensorflow.org/tutorials/recurrent ），将帮助您。

In [30]:
with graph.as_default():
    # Your basic LSTM cell
    lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    
    # Add dropout to the cell
    drop = tf.contrib.rnn.DropoutWrapper(lstm,output_keep_prob=keep_prob)
    
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([drop]*lstm_layers)
    
    # Getting an initial state of all zeros
    initial_state = cell.zero_state(batch_size, tf.float32)

### RNN forward pass

<img src="assets/network_diagram.png" width=400px>


现在我们需要通过RNN节点实际运行数据。您可以使用[`tf.nn.dynamic_rnn`]（https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn）来执行此操作。您将通过您创建的RNN单元格（例如，我们的多层次LSTM“单元格”）以及对网络的输入。

```
outputs，final_state = tf.nn.dynamic_rnn（cell，inputs，initial_state = initial_state）
```

以上我创建了一个初始状态`initial_state`，传递给RNN。这是在连续的时间步长中在隐藏层之间传递的单元格状态。 `tf.nn.dynamic_rnn`为我们照顾大部分的工作。我们将我们的单元格和输入信息传递到单元格，然后它对我们进行展开和其他一切。它返回每个时间步的输出和隐藏层的final_state。

> **练习：**使用`tf.nn.dynamic_rnn`添加RNN中的前进路径。记住，我们实际上是从嵌入层“embed”传递向量。



In [31]:
with graph.as_default():
    outputs, final_state = tf.nn.dynamic_rnn(cell,embed,initial_state=initial_state)

### 输出

我们只关心最终输出，我们将使用它作为我们的情绪预测。 所以我们需要用`outputs [:, -1]`获取最后一个输出，计算出来的成本和`labels_`。

In [32]:
with graph.as_default():
    predictions = tf.contrib.layers.fully_connected(outputs[:, -1], 1, activation_fn=tf.sigmoid)
    cost = tf.losses.mean_squared_error(labels_, predictions)
    
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

### 验证准确性

这里我们可以添加几个节点来计算我们将在验证过程中使用的准确度。

In [33]:
with graph.as_default():
    correct_pred = tf.equal(tf.cast(tf.round(predictions), tf.int32), labels_)
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

### 批处理

这是从我们的数据返回批次的一个简单的功能。 首先，它会删除数据，使我们只有完整的批次。 然后，它遍历“x”和“y”数组，并从大小为[[batch_size]]的数组中返回分片。

In [34]:
def get_batches(x, y, batch_size=100):
    
    n_batches = len(x)//batch_size
    x, y = x[:n_batches*batch_size], y[:n_batches*batch_size]
    for ii in range(0, len(x), batch_size):
        yield x[ii:ii+batch_size], y[ii:ii+batch_size]

## 训练

以下是典型的训练代码。 如果你想自己做这个，请随意删除所有这些代码，并自己实现。 在运行此操作之前，请确保“checkpoints”目录存在。

In [35]:
epochs = 10

with graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    iteration = 1
    for e in range(epochs):
        state = sess.run(initial_state)
        
        for ii, (x, y) in enumerate(get_batches(train_x, train_y, batch_size), 1):
            feed = {inputs_: x,
                    labels_: y[:, None],
                    keep_prob: 0.5,
                    initial_state: state}
            loss, state, _ = sess.run([cost, final_state, optimizer], feed_dict=feed)
            
            if iteration%5==0:
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Train loss: {:.3f}".format(loss))

            if iteration%25==0:
                val_acc = []
                val_state = sess.run(cell.zero_state(batch_size, tf.float32))
                for x, y in get_batches(val_x, val_y, batch_size):
                    feed = {inputs_: x,
                            labels_: y[:, None],
                            keep_prob: 1,
                            initial_state: val_state}
                    batch_acc, val_state = sess.run([accuracy, final_state], feed_dict=feed)
                    val_acc.append(batch_acc)
                print("Val acc: {:.3f}".format(np.mean(val_acc)))
            iteration +=1
    saver.save(sess, "checkpoints/sentiment.ckpt")

Epoch: 0/10 Iteration: 5 Train loss: 0.237
Epoch: 0/10 Iteration: 10 Train loss: 0.244
Epoch: 0/10 Iteration: 15 Train loss: 0.211
Epoch: 0/10 Iteration: 20 Train loss: 0.232
Epoch: 0/10 Iteration: 25 Train loss: 0.229
Val acc: 0.613
Epoch: 0/10 Iteration: 30 Train loss: 0.232
Epoch: 0/10 Iteration: 35 Train loss: 0.221
Epoch: 0/10 Iteration: 40 Train loss: 0.200
Epoch: 1/10 Iteration: 45 Train loss: 0.160
Epoch: 1/10 Iteration: 50 Train loss: 0.179
Val acc: 0.770
Epoch: 1/10 Iteration: 55 Train loss: 0.136
Epoch: 1/10 Iteration: 60 Train loss: 0.176
Epoch: 1/10 Iteration: 65 Train loss: 0.157
Epoch: 1/10 Iteration: 70 Train loss: 0.148
Epoch: 1/10 Iteration: 75 Train loss: 0.118
Val acc: 0.764
Epoch: 1/10 Iteration: 80 Train loss: 0.117
Epoch: 2/10 Iteration: 85 Train loss: 0.133
Epoch: 2/10 Iteration: 90 Train loss: 0.129
Epoch: 2/10 Iteration: 95 Train loss: 0.104
Epoch: 2/10 Iteration: 100 Train loss: 0.095
Val acc: 0.796
Epoch: 2/10 Iteration: 105 Train loss: 0.116
Epoch: 2/10 Ite

## Testing

In [36]:
test_acc = []
with tf.Session(graph=graph) as sess:
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    test_state = sess.run(cell.zero_state(batch_size, tf.float32))
    for ii, (x, y) in enumerate(get_batches(test_x, test_y, batch_size), 1):
        feed = {inputs_: x,
                labels_: y[:, None],
                keep_prob: 1,
                initial_state: test_state}
        batch_acc, test_state = sess.run([accuracy, final_state], feed_dict=feed)
        test_acc.append(batch_acc)
    print("Test accuracy: {:.3f}".format(np.mean(test_acc)))

INFO:tensorflow:Restoring parameters from checkpoints/sentiment.ckpt
Test accuracy: 0.813
